In [ ]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
nn.Module()

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 32
PERCENTILE = 50

epsilon = .01
EPSILON_DECAY = .95

class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
    
    def forward(self, x):
        return self.net(x)

Episode = namedtuple('Episode', field_names = ['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names = ['observation', 'action'])

def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim = 1)
    
    while True:
        obs_v = torch.FloatTensor([obs])
        act_prob_v = sm(net(obs_v))
        act_probs = act_prob_v.data.numpy()[0]
        
        if np.random.random() < epsilon:
            action = np.random.choice(env.action_space.n)
        else:        
            action = np.random.choice(len(act_probs), p = act_probs)
            
        next_obs, reward, done, _ = env.step(action)
        
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation = obs, action = action))
        
        if done:
            batch.append(Episode(reward = episode_reward, steps = episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

def filter_batch(batch, percentile):
    rewards = list(map(lambda x: x.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    
    train_obs = []
    train_act = []
    
    for example in batch:
        if example.reward < reward_bound:
            continue
            
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))
        
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    return train_obs_v, train_act_v, reward_bound, reward_mean

if __name__ == '__main__':
    env = gym.make('CartPole-v0')
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n
    
    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params = net.parameters(), lr = 0.01)
    
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        
        print('{0}: loss = {1:.3f}, reward_mean = {2:,.2f}, reward_bound = {3:,.2f}'.format(iter_no, 
                                                                                            loss_v.item(), 
                                                                                            reward_m, 
                                                                                            reward_b))
        if reward_m > 199:
            print('solved')
            break

0: loss = 0.700, reward_mean = 23.16, reward_bound = 20.00
1: loss = 0.685, reward_mean = 23.28, reward_bound = 17.50
2: loss = 0.669, reward_mean = 28.03, reward_bound = 23.00
3: loss = 0.650, reward_mean = 37.91, reward_bound = 30.50
4: loss = 0.648, reward_mean = 38.44, reward_bound = 33.50
5: loss = 0.626, reward_mean = 42.97, reward_bound = 39.00
6: loss = 0.618, reward_mean = 53.22, reward_bound = 42.50
7: loss = 0.618, reward_mean = 48.56, reward_bound = 43.50
8: loss = 0.600, reward_mean = 56.19, reward_bound = 49.00
9: loss = 0.601, reward_mean = 63.56, reward_bound = 57.00
10: loss = 0.578, reward_mean = 60.25, reward_bound = 52.00
11: loss = 0.566, reward_mean = 64.03, reward_bound = 63.50
12: loss = 0.566, reward_mean = 66.19, reward_bound = 54.50
13: loss = 0.564, reward_mean = 68.94, reward_bound = 64.00
14: loss = 0.552, reward_mean = 76.94, reward_bound = 68.00
15: loss = 0.565, reward_mean = 69.38, reward_bound = 66.00
16: loss = 0.536, reward_mean = 79.00, reward_boun